In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
class Layer:
    def __init__(self, input_size, output_size):
        self.weight = np.random.randn(output_size, input_size) * 0.01
        self.bias = np.zeros((output_size, 1))
        print("Weights:")
        print(self.weight)
        print("Biases:")
        print(self.bias)
        
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    
    def MSE_loss(self, a, y):
        return (a - y) ^ 2, 2 * (a-y)
            
    def forward_propagation(self, X):
        self.Z = np.dot(self.W, X) + self.B
        self.A = self.sigmoid(self.Z)
        return self.A
    
    def back_propagation(self, dA, X):
        m = X.shape[1]
        
        
        return 

In [ ]:
Layer1 = Layer(3,2)
input_data = np.array([[1], [4], [7]])
print()
print(input_data)
print()
print("Forward Output")
print(Layer1.forward_propagation(input_data))
true_output = np.array([[1], [0]])
print()
print("Back Output")
print(Layer1.back_propagation(true_output, input_data))
print(input_data.shape[1])